In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
starter_data = pd.read_csv("weather.csv")
starter_data

,Unnamed: 0,city,lat,lon,temp,humidity,cloudiness,wind speed
0,0,Cayenne,4.9333,-52.3333,80.60,78,75,14.97
1,1,Nouadhibou,20.9310,-17.0347,62.60,82,0,18.41
2,2,Mayna,54.1135,47.6231,0.25,94,4,7.70
3,3,Pevek,69.7008,170.3133,-13.72,88,91,3.49
4,4,Albany,42.6001,-73.9662,18.72,37,100,2.55
...,...,...,...,...,...,...,...,...
561,561,Richards Bay,-28.7830,32.0377,75.04,95,100,2.80
562,562,Husavik,66.0449,-17.3389,37.40,70,40,21.85
563,563,Nizwá,22.9333,57.5333,56.07,61,0,2.10
564,564,Teknāf,20.8624,92.3058,68.36,51,0,5.46


In [3]:
gmaps.configure(api_key=g_key)

In [13]:
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom_level': 10
}
fig = gmaps.figure(layout=figure_layout)

In [5]:
locations = starter_data[["lat", "lon"]]
humidity = starter_data["humidity"]


In [6]:
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [7]:
#df_filtered = df[df['Age'] >= 25] 
#df.drop(df[df['Age'] < 25].index, inplace = True) 
refined_data = starter_data[starter_data['temp'] <= 80]
refined_data = refined_data[refined_data['temp'] >= 70]
refined_data = refined_data[refined_data['wind speed'] < 10]
refined_data = refined_data[refined_data['cloudiness'] == 0]
refined_data

,Unnamed: 0,city,lat,lon,temp,humidity,cloudiness,wind speed
19,19,San Ignacio,17.1561,-89.0714,78.80,78,0,9.22
23,23,Busselton,-33.6500,115.3333,79.81,48,0,5.88
24,24,New Norfolk,-42.7826,147.0587,70.70,35,0,1.99
89,89,Qandala,11.4720,49.8728,72.32,53,0,3.96
148,148,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91
178,178,Lafiagi,8.8667,5.4167,75.79,65,0,1.54
183,183,Harper,4.3750,-7.7169,78.69,88,0,8.10
228,228,Laï,9.3952,16.2989,72.39,20,0,3.69
232,232,Gayéri,12.6497,0.4931,73.90,17,0,7.85
275,275,Kolondiéba,11.0882,-6.8926,73.78,20,0,1.14


In [9]:
# Create hotel_df and add column "hotel name"
hotel_df = refined_data
hotel_df["Hotel Name"] = ""
# Set search parameters
hotel_params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}
for index,row in hotel_df.iterrows():
    # Get lat, lng
    lat = row["lat"]
    lon = row["lon"]
    # change location each iteration while leaving original params in place
    hotel_params["location"] = f"{lat},{lon}"
    # Create query, make request, and store in json
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_response = requests.get(base_url, params=hotel_params)
    hotels = hotel_response.json()
    #Try to gather hotel names, but skip for those with no hotels in the set radius
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        pass
# Display updated df
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,Unnamed: 0,city,lat,lon,temp,humidity,cloudiness,wind speed,Hotel Name
19,19,San Ignacio,17.1561,-89.0714,78.80,78,0,9.22,Rainforest Haven Inn
23,23,Busselton,-33.6500,115.3333,79.81,48,0,5.88,Observatory Guest House
24,24,New Norfolk,-42.7826,147.0587,70.70,35,0,1.99,The Shingles Riverside Cottages
148,148,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91,39 On Nile Guest House
178,178,Lafiagi,8.8667,5.4167,75.79,65,0,1.54,Bus group international
183,183,Harper,4.3750,-7.7169,78.69,88,0,8.10,Screensaver
275,275,Kolondiéba,11.0882,-6.8926,73.78,20,0,1.14,Hotel Dakan
325,325,Am Timan,11.0297,20.2827,72.39,21,0,5.10,Palais du chef Canton Salamt
340,340,Manga,11.6636,-1.0731,73.71,18,0,6.22,Auberge du 11 décembre de la cité CNSS à Manga
346,346,Malanville,11.8685,3.3899,73.92,17,0,6.64,DEVO HOTEL


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [11]:
marker = gmaps.marker_layer(locations)

In [14]:
# Create a combined map
fig = gmaps.figure(layout=figure_layout)

fig.add_layer(heat_layer)
fig.add_layer(marker)


fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…